In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

2.13.0


In [4]:
# import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [6]:
dataset.head()  

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
X = dataset.iloc[:, 3:13]
y= dataset.iloc[:, 13]

In [8]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [9]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [10]:
## Feature Engineering
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [11]:
X.drop(['Geography','Gender'], axis=1, inplace=True)

In [12]:
X = pd.concat([X,geography,gender],axis=1)

In [13]:
from sklearn.model_selection import train_test_split   

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
# feature skaling
from sklearn.preprocessing import StandardScaler

In [16]:
sc = StandardScaler()

In [17]:
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [18]:
x_train.shape

(8000, 11)

In [19]:
x_test.shape

(2000, 11)

In [20]:
# Lets create the ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU,ReLU
from tensorflow.keras.layers import Dropout

In [21]:
# Initializing the ANN
classifier = Sequential()  

In [22]:
# adding the input layer
classifier.add(Dense(units=11, activation="relu"))

In [23]:
# first hidden layer
classifier.add(Dense(units=7, activation='ReLU'))

In [24]:
# second hidden layer
classifier.add(Dense(units=6, activation='ReLU'))

In [25]:
# adding the outout layer
classifier.add(Dense(units=1, activation='sigmoid'))

In [28]:
# compiling the ANN
classifier.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [30]:
model_history = classifier.fit(x_train, y_train, validation_split=0.33, batch_size=10, epochs=1000, callbacks=early_stopping)

Epoch 1/1000
536/536 [==============================] - 4s 5ms/step - loss: 0.4209 - accuracy: 0.8203 - val_loss: 0.3880 - val_accuracy: 0.8391
Epoch 2/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3611 - accuracy: 0.8541 - val_loss: 0.3677 - val_accuracy: 0.8493
Epoch 3/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3564 - accuracy: 0.8558 - val_loss: 0.3655 - val_accuracy: 0.8516
Epoch 4/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3498 - accuracy: 0.8576 - val_loss: 0.3685 - val_accuracy: 0.8489
Epoch 5/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3497 - accuracy: 0.8561 - val_loss: 0.3649 - val_accuracy: 0.8531
Epoch 6/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3480 - accuracy: 0.8572 - val_loss: 0.3665 - val_accuracy: 0.8527
Epoch 7/1000
536/536 [==============================] - 2s 3ms/step - loss: 0.3449 - accuracy: 0.8606 - val_loss: 0.3681 - val_accuracy:

In [29]:
import tensorflow as tf
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
      patience=20,
      verbose=1,
      mode='auto',
      baseline=None,
      restore_best_weights=False,
      )

In [31]:
model_history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [32]:
y_pred = classifier.predict(x_test)
y_pred = y_pred >= 0.5

63/63 [==============================] - 0s 2ms/step


In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)   
print(cm)

[[1482  113]
 [ 175  230]]


In [34]:
accuracy_score(y_test, y_pred)

0.856

In [35]:
## get the weights 
classifier.get_weights()

[array([[ 2.38121122e-01,  3.43131237e-02,  8.20957839e-01,
         -1.20676629e-01, -1.30164310e-01, -3.17815155e-01,
          8.13189149e-01,  3.31643730e-01,  5.17780900e-01,
          3.72638553e-01, -5.36876380e-01],
        [-2.91446060e-01, -2.99100232e+00, -8.21852088e-01,
          2.90701365e+00, -2.02105656e-01,  2.01988637e-01,
         -2.39908838e+00,  1.54784203e+00, -1.85943472e+00,
         -9.78731394e-01,  3.59820533e+00],
        [ 7.90708899e-01,  3.08239281e-01,  4.81801890e-02,
          1.38485610e-01,  1.27197433e+00,  7.89526820e-01,
         -5.83963871e-01, -2.22748280e-01,  1.20700017e-01,
          2.40387321e+00,  1.13949291e-01],
        [-1.94554758e+00,  2.75933802e-01,  2.74756253e-01,
          3.02978933e-01,  5.80303490e-01, -1.88419759e+00,
         -7.27032721e-01,  2.72965312e+00, -1.13400447e+00,
         -1.15070415e+00, -2.37514329e+00],
        [-5.97710550e-01,  1.06613648e+00, -9.35135901e-01,
          2.04019368e-01, -1.64080286e+00,  

In [ ]:
## add dropout layer
# classifier.add(Dropout(0.3))